In [ ]:
import os
import json
import numpy as np
from PIL import Image

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj) 
    
def save_to_json(dic,target_dir):
    dumped = json.dumps(dic, cls=NumpyEncoder)  
    file = open(target_dir, 'w')  
    json.dump(dumped, file)
    file.close()

def convert_str_list(pose):
    pose_sequence=[]
    for index in sorted(pose.keys()):
        dance_array = np.empty([2,17])
        for i in range(2):
            for j in range(17):
                dance_array[i][j]=pose[index][j][1][i]
        #print(dance_array)
        pose_sequence.append(dance_array)
        #print(dance_array)
    return pose_sequence
    
def overlap_get(target,overlapping,target_len,cat):
    N,C,V=target.shape
    x=[]
    label=[]
    if N<target_len:
        return x,label
    number=(N-target_len)//overlapping
    for i in range(number+1):
        slice=target[i*overlapping:(i*overlapping+50)]
        x.append(slice)
        label.append(cat)
    return x,label


In [ ]:
# you should change the imgPath to the download img path
txtList=['ballet', 'break', 'cha', 'flamenco', 'foxtrot', 'jive', 'latin', 'pasodoble', 'quickstep', 'rumba', 'samba', 'square', 'swing', 'tango', 'tap', 'waltz']
imgPath="/mnt/external4/densepose/rgb"

In [ ]:
for index in txtList:
    mainpath=os.path.join("/mnt/external4/densepose/txt",index)
    mainList= os.listdir(mainpath)
    mainImg=os.path.join(imgPath,index)
    mainList.sort()
    dance_dataset={}
    img_dataset={}
    c_name=""
    #c_num="0001"
    number=0
    for file_name in mainList:
        #print(file_name)
        number+=1
        path=os.path.join(mainpath,file_name)
        
        (file,extension) = os.path.splitext(file_name)
    
        split=file.split("_")
        name=split[0]
        if name =="":
            name=split[1]
        num=split[-1]
        if c_name != name:
            try:
                img = Image.open(os.path.join(mainImg,file+".png"))
                imgSize = img.size #图片的长和宽
            except FileNotFoundError:
                print ("Error: 没有找到文件或读取文件失败")
                imgSize = (0,0)
            #img = Image.open(os.path.join(mainImg,file+".png"))
            
            img_dataset[name]=list(imgSize)
            #dance_dataset[name]={}
            c_name=name
            
        
            
    json_file=index+".json"
    json_path=os.path.join("/mnt/external4/xuanchi/GIT_Img",json_file)
    with open(json_path, 'w') as f:
        json.dump(img_dataset, f)
    print(json_file+" is ok")